## In this kernel we convert LEVEL5 Lyft data (NuScenes format) to KITTI format, which is usually used in public repositories. After this you can search for repos, that solve KITTI 3d-detection task.

## Thanks to https://www.kaggle.com/stalkermustang/converting-lyft-dataset-to-kitty-format
### This above version by user - stalkermustang was not working for me so made little changes below

In [1]:
!pip install nuscenes-devkit

In [1]:
from pathlib import Path
from PIL import Image

In [2]:
# dir with all input data from Kaggle
INP_DIR = Path('/media/chris/Workspace/Dataset/3d-object-detection-for-autonomous-vehicles')
print(INP_DIR)

/media/chris/Workspace/Dataset/3d-object-detection-for-autonomous-vehicles


In [3]:
# dir with index json tables (scenes, categories, logs, etc...)
TABLES_DIR = INP_DIR.joinpath('train_data')

In [4]:
# Adjust the dataroot parameter below to point to your local dataset path.
# The correct dataset path contains at least the following four folders (or similar): images, lidar, maps
!ln -s {INP_DIR}/train_images images
!ln -s {INP_DIR}/train_maps maps
!ln -s {INP_DIR}/train_lidar lidar

ln: failed to create symbolic link 'images/train_images': File exists
ln: failed to create symbolic link 'maps/train_maps': File exists
ln: failed to create symbolic link 'lidar/train_lidar': File exists


In [5]:
DATA_DIR = Path().absolute() 
# Empty init equals '.'.
# We use this because we link train dirs to current dir (cell above)

In [6]:
# dir to write KITTY-style dataset
STORE_DIR = DATA_DIR.joinpath('kitti_format')

In [8]:
!pip install -U git+https://github.com/lyft/nuscenes-devkit

  Cloning https://github.com/lyft/nuscenes-devkit to /tmp/pip-req-build-n_nnin22
  Running command git clone --filter=blob:none --quiet https://github.com/lyft/nuscenes-devkit /tmp/pip-req-build-n_nnin22
  Resolved https://github.com/lyft/nuscenes-devkit to commit 49c36da0a85da6bc9e8f2a39d5d967311cd75069
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
import lyft_dataset_sdk
import lyft_dataset_sdk.utils.export_kitti

In [8]:
!python -m lyft_dataset_sdk.utils.export_kitti nuscenes_gt_to_kitti -h

INFO: Showing help with the command 'export_kitti.py nuscenes_gt_to_kitti -- --help'.

NAME
    export_kitti.py nuscenes_gt_to_kitti - Converts nuScenes GT formatted annotations to KITTI format.

SYNOPSIS
    export_kitti.py nuscenes_gt_to_kitti LYFT_DATAROOT TABLE_FOLDER <flags>

DESCRIPTION
    Converts nuScenes GT formatted annotations to KITTI format.

POSITIONAL ARGUMENTS
    LYFT_DATAROOT
        Type: str
        folder with tables (json files).
    TABLE_FOLDER
        Type: str
        folder with tables (json files).

FLAGS
    -l, --lidar_name=LIDAR_NAME
        Type: str
        Default: 'LIDAR_TOP'
        Name of the lidar sensor. Only one lidar allowed at this moment.
    -g, --get_all_detections=GET_ALL_DETECTIONS
        Type: bool
        Default: False
        If True, will write all bboxes in PointCloud and use only FrontCamera.
    -p, --parallel_n_jobs=PARALLEL_N_JOBS
        Type: int
        Default: 4
        Number of threads to parralel processing.
    -s, --

In [9]:
# convertation to KITTY-format
# samples count: 22680是training全部
# samples count: 7480是training部分
# samples count: 7518是testing的部分
!python -m lyft_dataset_sdk.utils.export_kitti nuscenes_gt_to_kitti \
        --lyft_dataroot {DATA_DIR} \
        --table_folder {TABLES_DIR} \
        --samples_count 7480 \
        --parallel_n_jobs 20 \
        --get_all_detections True \
        --store_dir {STORE_DIR}

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 6.3 seconds.
Reverse indexing ...
Done reverse indexing in 1.9 seconds.
 73%|████████████████████████████▍          | 5460/7480 [11:08<05:32,  6.07it/s]Traceback (most recent call last):
  File "/home/chris/anaconda3/envs/perception/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/chris/anaconda3/envs/perception/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/chris/anaconda3/envs/perception/lib/python3.9/site-packages/lyft_dataset_sdk/utils/export_kitti.py", line 337, in <module>
    fire.Fire(KittiConverter)
  File "/home/chris/anaconda3/envs/perception/lib/python3.9/site-packages/fire/core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_

In [12]:
# check created (converted) files. velodyne = LiDAR poinclouds data (in binary) host-a011_cam0_1233090652702363606
!ls {STORE_DIR}/velodyne | head -2

In [13]:
# render converted data for check. Currently don't support multithreading :(
!python -m lyft_dataset_sdk.utils.export_kitti render_kitti \
        --store_dir {STORE_DIR}

Rendering 3d boxes projected from 3d KITTI format
0it [00:00, ?it/s]


In [14]:
# Script above write images to 'render' folder
# in store_dir (where we have converted dataset)
RENDER_DIR = STORE_DIR.joinpath('render')

In [15]:
# get all rendered files
all_renders = list(RENDER_DIR.glob('*'))
all_renders.sort()

In [16]:
# render radar data (bird view) and camera data with bboxes

In [17]:
#Image.open(all_renders[0])

In [18]:
#Image.open(all_renders[1])

In [19]:
!ls ./kitti_format

calib  image_2	label_2  render  velodyne


In [20]:
!cat ./kitti_format/label_2/095d5bb88eb9cdd223b90d2a1475c0cf2f4b4c2a8aca82ba0ae51f6fba540440.txt

cat: ./kitti_format/label_2/095d5bb88eb9cdd223b90d2a1475c0cf2f4b4c2a8aca82ba0ae51f6fba540440.txt: No such file or directory


In [21]:
!ls kitti_format/image_2/

In [22]:
path = 'kitti_format/image_2/095d5bb88eb9cdd223b90d2a1475c0cf2f4b4c2a8aca82ba0ae51f6fba540440.png'
from IPython.display import Image
Image(path)

FileNotFoundError: No such file or directory: 'kitti_format/image_2/095d5bb88eb9cdd223b90d2a1475c0cf2f4b4c2a8aca82ba0ae51f6fba540440.png'

FileNotFoundError: No such file or directory: 'kitti_format/image_2/095d5bb88eb9cdd223b90d2a1475c0cf2f4b4c2a8aca82ba0ae51f6fba540440.png'

<IPython.core.display.Image object>

In [ ]:
!ls ./kitti_format/label_2/ | wc -l

In [ ]:
!ls ./kitti_format/image_2/ | wc -l

In [ ]:
!ls ./kitti_format/render | wc -l

### 40 denotes, 20 samples were taken - which was then splitted into 20 lidar + 20 camera images